# Data Prepatation
- this one uses the github repo of data, not the google sheet

In [3]:
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import os
import git
import pickle
import os.path
from datetime import datetime, date, time 
from time import strftime
import pyarrow
import json
from dateutil.parser import parse

pd.options.display.max_rows = 1000

In [14]:
str(datetime.date(datetime.now()))

'2020-02-15'

In [21]:
'''
Download github repo and extract csv's

https://github.com/CSSEGISandData/2019-nCoV
'''

REPO = 'https://github.com/CSSEGISandData/COVID-19.git'
TMP_FOLDER = '/tmp/corona/'
TMP_GIT = os.path.join(TMP_FOLDER, 'COVID-19')
DATA = os.path.join(TMP_GIT, 'csse_covid_19_data/csse_covid_19_daily_reports/')


def clone_repo(TMP_FOLDER, REPO):
    print('Cloning Data Repo...')
    git.Git(TMP_FOLDER).clone(REPO)

# Create Tmp Folder
if not os.path.isdir(TMP_FOLDER):
    print('Creating folder...')
    print('...', TMP_FOLDER)
    os.mkdir(TMP_FOLDER)

#Check if repo exists
if not os.path.isdir(TMP_GIT):
    clone_repo(TMP_FOLDER, REPO)
else:
    #get up to date repo
    print('Deleting out of date repo...')
    os.system('rm -rf ' + str(TMP_GIT))
    clone_repo(TMP_FOLDER, REPO)

print('Done!')

Deleting out of date repo...
Cloning Data Repo...
Done!


In [22]:
def clean_sheet_names(new_ranges):
    '''
    Get rid of the duplicate sheets, only take the sheets from the 
    latest point in the day
    '''
    indices = []
    
    # Remove all sheets that dont have a numeric header
    numeric_sheets = [x for x in new_ranges if re.search(r'\d', x)]
        
#     #split the names to just get the date
#     clean_new_ranges = []
#     for i, x in enumerate(numeric_sheets):
#         clean_new_ranges.append(x.split('_')[0])    
    
    return numeric_sheets

In [23]:
'''
For assigning date by the time sheet name
'''

def clean_last_updates(last_update):
    date = parse(str(last_update).split(' ')[0]).strftime("%Y-%m-%d")
    time = parse(str(last_update).split(' ')[1]).strftime('%H:%M:%S')
    parsed_date = str(date) + ' ' + str(time)

    return parsed_date

def get_date(last_update):
    return parse(str(last_update).split(' ')[0]).strftime("%Y-%m-%d")

In [24]:
sheets = os.listdir(DATA)

# Clean the result to the sheet tabs we want
cleaned_sheets = clean_sheet_names(sorted(sheets, reverse=True))

keep_cols = ['Confirmed', 'Country/Region', 'Deaths', 'Last Update', 'Province/State', 'Recovered']
numeric_cols = ['Confirmed', 'Deaths', 'Recovered']
all_csv = []
# Import all CSV's
for file in sorted(sheets):
    if 'csv' in file:
        print('...', file)
        tmp_df = pd.read_csv(os.path.join(DATA, file), index_col=None, header=0, parse_dates=['Last Update'])
        tmp_df = tmp_df[keep_cols]
        tmp_df[numeric_cols] = tmp_df[numeric_cols].fillna(0)
        tmp_df[numeric_cols] = tmp_df[numeric_cols].astype(int)
        tmp_df['Province/State'].fillna(tmp_df['Country/Region'], inplace=True)
        
        tmp_df['Last Update'] = tmp_df['Last Update'].apply(clean_last_updates)
        tmp_df['date'] = tmp_df['Last Update'].apply(get_date)
        
        all_csv.append(tmp_df)
    
df_raw = pd.concat(all_csv, axis=0, ignore_index=True, sort=True)
df_raw = df_raw.sort_values(by=['Last Update'])


... 01-22-2020.csv
... 01-23-2020.csv
... 01-24-2020.csv
... 01-25-2020.csv
... 01-26-2020.csv
... 01-27-2020.csv
... 01-28-2020.csv
... 01-29-2020.csv
... 01-30-2020.csv
... 01-31-2020.csv
... 02-01-2020.csv
... 02-02-2020.csv
... 02-03-2020.csv
... 02-04-2020.csv
... 02-05-2020.csv
... 02-06-2020.csv
... 02-07-2020.csv
... 02-08-2020.csv
... 02-09-2020.csv
... 02-10-2020.csv
... 02-11-2020.csv
... 02-12-2020.csv
... 02-13-2020.csv
... 02-14-2020.csv


In [27]:
df_raw.tail(10)

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,date
1417,1184,Mainland China,11,2020-02-14 12:53:02,Henan,357,2020-02-14
1426,419,Mainland China,11,2020-02-14 13:23:07,Heilongjiang,47,2020-02-14
1418,1155,Mainland China,0,2020-02-14 14:13:15,Zhejiang,403,2020-02-14
1432,226,Mainland China,2,2020-02-14 15:23:02,Guangxi,36,2020-02-14
1435,159,Mainland China,4,2020-02-14 15:23:02,Hainan,43,2020-02-14
1439,119,Mainland China,1,2020-02-14 15:23:02,Liaoning,29,2020-02-14
1437,127,Mainland China,0,2020-02-14 23:23:04,Shanxi,38,2020-02-14
1415,54406,Mainland China,1457,2020-02-14 23:33:02,Hubei,4774,2020-02-14
1423,537,Mainland China,5,2020-02-14 23:33:02,Chongqing,152,2020-02-14
1477,1,Egypt,0,2020-02-14 23:53:02,Egypt,0,2020-02-14


In [28]:
# sheets need to be sorted by date value
print('Sorting by datetime...')
if df[df.date.max() == str(datetime.date(datetime.now()))]:
    df = df[df.date != df.date.max()]
else:
    df = df[df.date != str(datetime.date(datetime.now()))]

Sorting by datetime...


NameError: name 'df' is not defined

In [30]:
def drop_duplicates(df_raw):
    '''
    Take the max date value for each province for a given date
    '''
    days_list = []
    
    for datetime in df_raw.date.unique():
        tmp_df = df_raw[df_raw.date == datetime]
        tmp_df = tmp_df.sort_values(['Last Update']).drop_duplicates('Province/State', keep='last')
        days_list.append(tmp_df)

    return days_list
        
frames = drop_duplicates(df_raw)
df = pd.concat(frames, axis=0, ignore_index=True, sort=True)

In [31]:
df2 = df.copy()

In [32]:
df2

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered,date
0,1,Mainland China,0,2020-01-22 17:00:00,Anhui,0,2020-01-22
1,0,Mainland China,0,2020-01-22 17:00:00,Jilin,0,2020-01-22
2,14,Mainland China,0,2020-01-22 17:00:00,Beijing,0,2020-01-22
3,6,Mainland China,0,2020-01-22 17:00:00,Chongqing,0,2020-01-22
4,1,Mainland China,0,2020-01-22 17:00:00,Fujian,0,2020-01-22
...,...,...,...,...,...,...,...
1048,119,Mainland China,1,2020-02-14 15:23:02,Liaoning,29,2020-02-14
1049,127,Mainland China,0,2020-02-14 23:23:04,Shanxi,38,2020-02-14
1050,54406,Mainland China,1457,2020-02-14 23:33:02,Hubei,4774,2020-02-14
1051,537,Mainland China,5,2020-02-14 23:33:02,Chongqing,152,2020-02-14


In [33]:
# Now that we have all the data we now need to clean it 
# - Fill null values
# - remore suspected values
# - change column names
def clean_data(tmp_df):
    if 'Demised' in tmp_df.columns:
        tmp_df.rename(columns={'Demised':'Deaths'}, inplace=True)

    if 'Country/Region' in tmp_df.columns:
        tmp_df.rename(columns={'Country/Region':'country'}, inplace=True)
    
    if 'Province/State' in tmp_df.columns:
        tmp_df.rename(columns={'Province/State':'province'}, inplace=True)
      
    if 'Last Update' in tmp_df.columns:
        tmp_df.rename(columns={'Last Update':'datetime'}, inplace=True)
        
    if 'Suspected' in tmp_df.columns:
        tmp_df = tmp_df.drop(columns='Suspected')

    #Lower case all col names
    tmp_df.columns = map(str.lower, tmp_df.columns)    

    return tmp_df

In [34]:
print('Cleaning dataframes...')
df  = clean_data(df2)

Cleaning dataframes...


In [37]:
# df.groupby(['country', 'province']).confirmed.count()

current_date = str(datetime.date(datetime.now()))


In [38]:
# sheets need to be sorted by date value
print('Sorting by datetime...')
if df.date.max() == current_date:
    df = df[df.date != df.date.max()]
else:
    df = df[df.date != current_date]


df = df.sort_values('datetime')

Sorting by datetime...


In [39]:
df.tail()

,confirmed,country,deaths,datetime,province,recovered,date
1048,119,Mainland China,1,2020-02-14 15:23:02,Liaoning,29,2020-02-14
1049,127,Mainland China,0,2020-02-14 23:23:04,Shanxi,38,2020-02-14
1051,537,Mainland China,5,2020-02-14 23:33:02,Chongqing,152,2020-02-14
1050,54406,Mainland China,1457,2020-02-14 23:33:02,Hubei,4774,2020-02-14
1052,1,Egypt,0,2020-02-14 23:53:02,Egypt,0,2020-02-14


In [160]:
df[df.country == 'Australia']

,confirmed,country,deaths,datetime,province,recovered,date
105,0,Australia,0,2020-01-23 12:00:00,Australia,0,2020-01-23
189,4,Australia,0,2020-01-25 12:00:00,Australia,0,2020-01-25
237,4,Australia,0,2020-01-26 23:00:00,Australia,0,2020-01-26
243,5,Australia,0,2020-01-27 09:00:00,Australia,0,2020-01-27
293,4,Australia,0,2020-01-27 20:30:00,New South Wales,0,2020-01-27
284,1,Australia,0,2020-01-27 20:30:00,Victoria,0,2020-01-27
342,4,Australia,0,2020-01-28 23:00:00,New South Wales,0,2020-01-28
343,1,Australia,0,2020-01-28 23:00:00,Victoria,0,2020-01-28
396,4,Australia,0,2020-01-29 21:00:00,New South Wales,0,2020-01-29
397,2,Australia,0,2020-01-29 21:00:00,Victoria,0,2020-01-29


In [27]:

'''
Get the difference of the sum totals for each
date and plot them on a trendline graph
'''
def get_new_cases(final_df, col):
    diff_list = []
    tmp_df_list = []
    df = final_df.copy()
    
#     for column in ['confirmed', 'deaths', 'recovered']:
#         df[column] = df[column].replace('', 0).astype(int)

    for i, day in enumerate(df.date.unique()):    
        tmp_df = df[df.date == day]
        tmp_df_list.append(tmp_df[col].sum())
        
        if i == 0:
            diff_list.append(tmp_df[col].sum())
        else:
            diff_list.append(tmp_df[col].sum() - tmp_df_list[i-1])
        
    return diff_list

print('Calculating dataframe for new cases...')
daily_cases_df = pd.DataFrame([])
daily_cases_df['new_confirmed_cases'] = get_new_cases(df, 'confirmed')
daily_cases_df['new_deaths'] = get_new_cases(df, 'deaths')
daily_cases_df['new_recoveries'] = get_new_cases(df, 'recovered')
daily_cases_df['date'] = df.date.unique()


Calculating dataframe for new cases...


In [22]:
for column in ['confirmed', 'deaths', 'recovered']:
    df[column] = df[column].replace('', 0).astype(int)

current_infected = pd.DataFrame([])
current_infected['currently_infected'] = df.groupby('date').confirmed.sum() - (df.groupby('date').deaths.sum() + df.groupby('date').recovered.sum())

In [23]:
current_infected['delta'] = (current_infected['currently_infected'] - df.groupby('date').confirmed.sum())

In [24]:
daily_cases_df = pd.merge(daily_cases_df, current_infected, how='outer', on='date')

In [25]:
daily_cases_df

,new_confirmed_cases,new_deaths,new_recoveries,date,currently_infected,delta
0,332,6,25,2020-01-21,301,-31
1,223,11,3,2020-01-22,510,-45
2,98,1,2,2020-01-23,605,-48
3,288,8,6,2020-01-24,879,-62
4,1078,30,13,2020-01-25,1914,-105
5,775,24,5,2020-01-26,2660,-134
6,1684,27,9,2020-01-27,4308,-170
7,1579,25,47,2020-01-28,5815,-242
8,1740,38,23,2020-01-29,7494,-303
9,1979,43,54,2020-01-30,9376,-400


In [19]:
#Create date of extraction folder
save_dir  = '../data/' + str(datetime.date(datetime.now()))

print('Saving to data subdirectory...')
print('...', save_dir)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

Saving to data subdirectory...
... ../data/2020-02-12


In [20]:
print('Saving...')
file_name = 'agg_data_{}.parquet.gzip'.format(datetime.date(datetime.now()))
df.astype(str).to_parquet(os.path.join(save_dir, file_name), compression='gzip')
print('...', file_name)


csv_file_name = 'agg_data_{}.csv'.format(datetime.date(datetime.now()))
df.astype(str).to_csv(os.path.join(save_dir, csv_file_name))
print('...', csv_file_name)


new_case_file_name = 'trend_{}.csv'.format(datetime.date(datetime.now()))
daily_cases_df.astype(str).to_csv(os.path.join(save_dir, new_case_file_name))
print('...', new_case_file_name)

print('Done!')

Saving...
... agg_data_2020-02-12.parquet.gzip
... agg_data_2020-02-12.csv
... trend_2020-02-12.csv
Done!


In [28]:
df[df.date == '2020-02-12']

,confirmed,country,deaths,datetime,province,recovered,date
987,28,South Korea,0,2020-02-12 00:03:12,South Korea,4,2020-02-12
988,1,Finland,0,2020-02-12 00:03:12,Finland,1,2020-02-12
989,251,Mainland China,2,2020-02-12 00:03:12,Hebei,48,2020-02-12
991,378,Mainland China,8,2020-02-12 00:13:27,Heilongjiang,28,2020-02-12
990,306,Mainland China,1,2020-02-12 00:13:27,Shanghai,53,2020-02-12
992,151,Mainland China,3,2020-02-12 00:33:02,Hainan,20,2020-02-12
993,8,United Arab Emirates,0,2020-02-12 00:33:02,United Arab Emirates,1,2020-02-12
994,222,Mainland China,1,2020-02-12 00:33:02,Guangxi,31,2020-02-12
995,116,Mainland China,0,2020-02-12 00:43:02,Liaoning,19,2020-02-12
996,1135,Mainland China,8,2020-02-12 00:43:02,Henan,231,2020-02-12
